In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pingouin
import seaborn as sns
import scipy
import statistics
import os
from bioinfokit.analys import stat
from statannotations.Annotator import Annotator

# CHANGE THIS VARIABLE TO REFLECT WHERE YOUR HBN BIDS DATA LIVE ()
bids_dir = '/om4/group/gablab/data/hbn_bids/' # Path should end with a '/'

# THESE VARIABLES SHOULD NOT CHANGE IF THE ANALYSIS WAS RUN ACCORDING TO INSTRUCTIONS
code_dir = bids_dir+'code/'
derivatives_dir = bids_dir+'derivatives/'
pod2_dir = derivatives_dir+'qsiprep/'
fba_dir = derivatives_dir+'fba/'
freesurfer_dir = derivatives_dir+'freesurfer/'
out_variable_dir = os.getcwd()+'/output_variables/'

# Load DataFrame made in step 3 notebook
df = pd.read_pickle(out_variable_dir+'df.pkl')

## TR vs RD Stats (Table 1)

In [ ]:
#Get the mean, sum, and std of stats broken down by groups
mean_stats_tr_rd = df.groupby('GROUP').mean()
sum_stats_tr_rd = df.groupby('GROUP').sum()
sem_stats_tr_rd = df.groupby('GROUP').sem()
# Get indices of TR and RD participants
inds_tr = (df['GROUP']=='TR')
inds_rd = (df['GROUP']=='RD')

# PRINT COHORT-WIDE STATISTICS
num_tr = sum(inds_tr)
num_rd = sum(inds_rd)
num_female_tr = sum(df[inds_tr]['SEX']=='F')
num_female_rd = sum(df[inds_rd]['SEX']=='F')
# Print sexes of participants
print('n TR:', num_tr, ', M (',num_tr - num_female_tr,')', 'F(',num_female_tr,')')
print('n RD:', num_rd, ', M (',num_rd - num_female_rd,')', 'F(',num_female_rd,')\n')
# Print metric-by-metric t-stats
for metric in ['AGE','EHI','SES','ICV','WISC_VSI','WISC_VCI','TOWRE','gFD','gFC','MOTION','N_CORR']:
    print('AVERAGE '+metric+':',np.nanmean(df[metric]),'(',scipy.stats.sem(df[metric],nan_policy='omit'),')')
    print('COMPARE '+metric+': TR MEAN (SD):',mean_stats_tr_rd[metric]['TR'],'(',sem_stats_tr_rd[metric]['TR'],'), RD MEAN (SD):', 
          mean_stats_tr_rd[metric]['RD'],'(',sem_stats_tr_rd[metric]['RD'],')')
    print(pingouin.ttest(x=np.asarray(df[metric][inds_tr]), y=np.asarray(df[metric][inds_rd])),'\n')

## Plot TOWRE Scores / Group Figure (Figure 2)

In [ ]:
x_label = 'SWE'
y_label = 'PDE'
hue = 'GROUP'
# Scatterplot

corr_plot = sns.jointplot(data=df, x=x_label, y=y_label,hue=hue,height=10,legend=True,
                         xlim=[min(df[x_label])-2,max(df[x_label])+2],
                          ylim=[min(df[y_label])-2,max(df[y_label])+2])
corr_plot.ax_joint.plot([85,85], [85,0], color='k',linestyle='--', linewidth = 2)
corr_plot.ax_joint.plot([0,85], [85,85], color='k',linestyle='--', linewidth = 2)
corr_plot.ax_joint.plot([90,90], [90,160], color='k',linestyle='--', linewidth = 2)
corr_plot.ax_joint.plot([160,90], [90,90], color='k',linestyle='--', linewidth = 2)
corr_plot.ax_joint.set_xlabel('TOWRE Sight Word Efficiency (Age-Standardized)',fontsize=20)
corr_plot.ax_joint.set_ylabel('TOWRE Phonemic Decoding Efficiency (Age-Standardized)',fontsize=20)


#plt.savefig(out_variable_dir+'towre.pdf',format='pdf')
plt.show()

## Plot Correlation Matrix of Continuous Phenotypic Variables (Figure 3)

In [ ]:
# Get dataframe with just covariates
sns.set(font_scale=1.4, font='Arial')
sns.set_style("white")

df_covars = df[['AGE','TOWRE','WISC_VCI','WISC_VSI','SES','ICV','MOTION','N_CORR','gFD','gFC']]
# Calculate correlations across all pairwise columns
corr = df_covars.corr(method='spearman')

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 15))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax = 1, vmin = -1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True, annot_kws={"fontsize":15})

#plt.savefig(out_variable_dir+'covars_corr.pdf',format='pdf')
plt.show()
pingouin.rcorr(df_covars,method='spearman',padjust='fdr_bh', pval_stars={0.00001: '***', 0.001: '**', 0.05: '*'})

## Differences Between Sites (Figure S1)

In [ ]:
# PRINT COHORT-WIDE STATISTICS
order = ['CBIC', 'CUNY', 'RU', 'SI'] # order of sites on plots
between = 'SITE' # Variable to run ANOVA over

for metric in ['AGE','EHI','SES','ICV','WISC_VCI','WISC_VSI','TOWRE','gFD','gFC','MOTION','N_CORR']:
    print('\n',metric)
    # run anova and get stats
    anova = pingouin.anova(data=df,dv=metric,between=between)
    fvalue = anova['F'][0]
    pvalue = anova['p-unc'][0]
    print('F-stat:',fvalue, ', p-val:',pvalue)
    
    # If anova is significant, run post-hoc t-tests
    if pvalue < 0.05:
        # run tukey-corrected multiple t-tests
        res = stat()
        res.tukey_hsd(df=df, res_var=metric, xfac_var=between, anova_model=metric+'~ C('+between+')')
        # create and annotate graph
        box_pairs = [(g1,g2) for g1,g2 in zip(res.tukey_summary['group1'],res.tukey_summary['group2'])]
        ax = sns.violinplot(data=df, x=between, y=metric, order=order)
        annotator = Annotator(ax, box_pairs, data=df, x=between, y=metric, order=order)
        annotator.configure(test='t-test_welch', text_format='star', loc='outside')
        annotator.apply_and_annotate()
        plt.show()


## M vs F Stats

In [ ]:
#Get the mean, sum, and std of stats broken down by sex
mean_stats_sex = df.groupby('SEX').mean()
sum_stats_sex = df.groupby('SEX').sum()
sem_stats_sex = df.groupby('SEX').sem()
# Get indices of M and F participants
inds_m = (df['SEX']=='M')
inds_f = (df['SEX']=='F')

# PRINT COHORT-WIDE STATISTICS
for metric in ['AGE','EHI','SES','ICV','WISC_VCI','WISC_VSI','TOWRE','gFD','gFC','MOTION','N_CORR']:
    print('COMPARE '+metric+': M MEAN (SD):',mean_stats_sex[metric]['M'],'(',sem_stats_sex[metric]['M'],'), F MEAN (SD):', 
          mean_stats_sex[metric]['F'],'(',sem_stats_sex[metric]['F'],')')
    print(pingouin.ttest(x=np.asarray(df[metric][inds_m]), y=np.asarray(df[metric][inds_f])),'\n')

## Compare Differences in Stats Between Handedness

In [ ]:
# PRINT COHORT-WIDE STATISTICS
order = ['L', 'A', 'R'] # order of sites on plots
between='HAND'

for metric in ['AGE','SES','ICV','WISC_VCI','WISC_VSI','TOWRE','gFD','gFC','MOTION','N_CORR']:
    print('\n',metric)
    # run anova and get stats
    anova = pingouin.anova(data=df,dv=metric,between=between)
    fvalue = anova['F'][0]
    pvalue = anova['p-unc'][0]
    print('F-stat:',fvalue, ', p-val:',pvalue)
    
    # If anova is significant, run post-hoc t-tests
    if pvalue < 0.05:
        # run tukey-corrected multiple t-tests
        res = stat()
        res.tukey_hsd(df=df, res_var=metric, xfac_var=between, anova_model=metric+'~ C('+between+')')
        # create and annotate graph
        box_pairs = [(g1,g2) for g1,g2 in zip(res.tukey_summary['group1'],res.tukey_summary['group2'])]
        ax = sns.violinplot(data=df, x=between, y=metric, order=order)
        annotator = Annotator(ax, box_pairs, data=df, x=between, y=metric, order=order)
        annotator.configure(test='t-test_welch', text_format='star', loc='outside')
        annotator.apply_and_annotate()
        plt.show()

## Extra: run your own correlations!

In [ ]:
# Add regression line
x_label='TOWRE'
y_label='WISC_VSI'
hue = 'GROUP'
corr_plot = sns.jointplot(data=df, x=x_label, y=y_label,hue=hue,height=10,legend=True,
                         xlim=[min(df[x_label])-2,max(df[x_label])+2],
                          ylim=[min(df[y_label])-2,max(df[y_label])+2])
sns.regplot(x=x_label, y=y_label, data=df, robust=True, ax=corr_plot.ax_joint, 
                scatter_kws={'s':0}, line_kws={"color":"black"})
#Compute correlation stats
corr = pingouin.corr(x=df[x_label], y=df[y_label], method='skipped')
print(corr)